In [5]:
! pip install requests
! pip install beautifulsoup4
! pip install openai
! pip install streamlit
! pip install scikit-learn

In [6]:
# prompt: code that will pull a html page from the web and phase is

import requests
from bs4 import BeautifulSoup

# Define the URL of the web page you want to scrape
url = "https://a.co/d/7Z6Ne4i"

# Send a GET request to the URL and get the response
response = requests.get(url)

# Check for successful response
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the paragraph elements on the page
    paragraphs = soup.find_all('p')

    # Print the text of each paragraph
    for paragraph in paragraphs:
        print(paragraph.text)
else:
    # Handle error
    print(f"Error fetching the web page: {response.status_code}")


MageGee Portable 60% Mechanical Gaming Keyboard, MK-Box LED Backlit Compact 68 Keys Mini Wired Office Keyboard with Red Switch for Windows Laptop PC Mac - Black/Grey
Share: 
Found a lower price? Let us know. Although we can't match every price reported, we'll use your feedback to ensure that our prices remain competitive.



 
 · Dimension: 30.8*10.1*3.9cm, 12.1*4*1.5in 
 · Net Weight: 570g,1.26lb 
 · Keys: 68 keys 
 · Switch: red switch 
 · Connection: detachable type-c wired 
 · Keycap: ergonomics ABS double-shot injection 
 · Lighting: LED blue backlit, press Fn+Ins control about 15 kinds light modes 

 

 

 
 To report an issue with this product or seller, click here. 
Customer Reviews, including Product Star Ratings help customers to learn more about the product and decide whether it is the right product for them.
To calculate the overall star rating and percentage breakdown by star, we don’t use a simple average. Instead, our system considers things like how recent a review is a

In [8]:

# Create a prompt that contains the paragraphs
prompt = """
extract list of features of this product for each features, categorized the persona that will use it.
response in json

[{
  "feature": <one of the features of the product>
  "personns": [
    <one of the persona that will need this feauture>,
    // add more here
  ]

}]
          """
for paragraph in paragraphs:
    prompt += paragraph.text + "\n"

# Call OpenAI with the prompt
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    stream=False,
)
print(stream.choices[0].message.content)

```json
[
    {
        "feature": "Dimension 30.8*10.1*3.9cm",
        "personas": [
            "Users who prefer a compact keyboard"
        ]
    },
    {
        "feature": "Net Weight: 570g,1.26lb",
        "personas": [
            "Users looking for a lightweight keyboard"
        ]
    },
    {
        "feature": "Keys: 68 keys",
        "personas": [
            "Gamers who prefer a minimalist keyboard setup"
        ]
    },
    {
        "feature": "Switch: red switch",
        "personas": [
            "Gamers looking for a responsive switch"
        ]
    },
    {
        "feature": "Connection: detachable type-c wired",
        "personas": [
            "Users who value convenience and portability"
        ]
    },
    {
        "feature": "Keycap: ergonomics ABS double-shot injection",
        "personas": [
            "Users looking for durable keycaps"
        ]
    },
    {
        "feature": "Lighting: LED blue backlit, press Fn+Ins control about 15 kinds light mode

In [9]:
import json

product_persons = json.loads(stream.choices[0].message.content.replace("json","").replace("```",""))

persons_list = []
for feature in product_persons:
  for person in feature["personas"]:
    persons_list.append(person)

print(persons_list)

['Users who prefer a compact keyboard', 'Users looking for a lightweight keyboard', 'Gamers who prefer a minimalist keyboard setup', 'Gamers looking for a responsive switch', 'Users who value convenience and portability', 'Users looking for durable keycaps', 'Users who want customizable lighting options']


In [12]:
from openai import OpenAI
import pandas as pd
client = OpenAI()

df = pd.DataFrame(data={"persona":persons_list})
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_embedding'] = df.persona.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df.to_csv('basic_persona_v1.csv', index=False)

In [120]:
def search_by_persona(input_text):
    import pandas as pd
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np
    import ast
    from numpy import dot
    from numpy.linalg import norm


    data = pd.read_csv('basic_persona_v1.csv')
    data["ada_embedding"] = data["ada_embedding"].apply(lambda x: ast.literal_eval(x))

    # Calculate cosine similarity
    input_embedding = get_embedding(input_text)


    # Calculate cosine similarity
    similarities = data['ada_embedding'].apply(lambda x:dot(x, input_embedding)/(norm(x)*norm(input_embedding)))

    return data.iloc[similarities.sort_values().index,:]['persona']


In [122]:
# prompt: get user input
name = input("Tell me about you self?")
print(f"User input is: {name}")
ordered_res = search_by_persona(name)
print(ordered_res)

User input is: I love taveling
3           Gamers looking for a responsive switch
5                Users looking for durable keycaps
1         Users looking for a lightweight keyboard
2    Gamers who prefer a minimalist keyboard setup
6     Users who want customizable lighting options
0              Users who prefer a compact keyboard
4      Users who value convenience and portability
Name: persona, dtype: object
